In [1]:
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

from ipynb.fs.full.ga_nonogram import *

import random
import numpy as np

In [2]:
def createIndividuals(nonogram, creator, n):
    rows, cols, row_blocks, col_blocks = getInfo(nonogram)
    individuals = []
    
    for i in range(n):
        geno = defineGenotype(cols)
        decoded = decodeGenotype(rows, geno)
        individual = getSolution(cols, decoded)
        individual = creator(individual)
        individuals.append(individual)
        
    return individuals

In [3]:
def evalOneMax(individual, nonogram):
    rows, cols, row_blocks, col_blocks = getInfo(nonogram)
    nonogram = getNonogramFromIndividual(rows, col_blocks, individual)
    solution_rows = getRowsFromNonogram(nonogram)

    result = sum(x != y for x, y in zip(rows, solution_rows))

    return (result,)

In [4]:
def cxTwoPoints(ind1, ind2):

    size = min(len(ind1), len(ind2))
    
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]

    return ind1, ind2

In [5]:
def mutShuffleIndexes(individual, indpb, nonogram):
    rows, cols, row_blocks, col_blocks = getInfo(nonogram)
    size = len(individual)

    for i in range(size):
        if random.random() < indpb:
            
            swap_indx = random.randint(0, size - 1)
            result1 = sum(x + y if idx == 0 else x + y + 1 for idx, (x, y) in enumerate(zip(individual[swap_indx], cols[i])))
            result2 = sum(x + y if idx == 0 else x + y + 1 for idx, (x, y) in enumerate(zip(individual[i], cols[swap_indx])))
                        
            if len(individual[swap_indx]) == len(individual[i]) and result1 <= 9 and result2 <= 9:
                individual[i], individual[swap_indx] = individual[swap_indx], individual[i]
                
            else:
                individual[i].reverse()
                            
    return individual,

In [6]:
def eaSimpleWithElitism(population, toolbox, cxpb, mutpb, ngen, stats=None, halloffame=None, verbose=__debug__):
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is None:
        raise ValueError("halloffame parameter must not be empty!")

    halloffame.update(population)
    hof_size = len(halloffame.items) if halloffame.items else 0

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    print('\n')
    if verbose:
        print(logbook.stream)

    for gen in range(1, ngen + 1):
        offspring = toolbox.select(population, len(population) - hof_size)
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        offspring.extend(halloffame.items)
        halloffame.update(offspring)
        population[:] = offspring
        
        prev_avg = logbook.select("avg")[-1]
        
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        
        if verbose:
            print(logbook.stream)
            
        if record.get("min") == 0 or round(record.get("avg"), 2) >= round(prev_avg, 2) :
            break

    return population, logbook